# Part 06 - Molecular dynamics example (and numpy)

Molecular dynamics takes particles at a specified initial condition (velocities and positions) and calculates their new position and velocity after a discrete time step $\Delta t$. We're not going into details here, but the algorithm for performing the update of a single particle $i$ can be derived from a Taylor series, 

\begin{align}
\vec{r}_i(t+\Delta t) &= \vec{r}_i(t) + \Delta t \frac{{\rm d}\vec{r}_i(t)}{{\rm d} t} + \frac{\Delta t^2}{2} \frac{{\rm d}^2\vec{r}_i(t)}{{\rm d} t^2}+\ldots
\\
&\approx \vec{r}(t) + \Delta t\,\vec{v}_i(t) + \frac{\Delta t^2}{2} \vec{a}_i(t)
\end{align}
where $\vec{r}_i$ is the position of a particle, $\vec{v}_i$ is its velocity, and $\vec{a}_i$ is its acceleration.
The velocity update is obtained from a similar approach,
\begin{align}
\vec{v}(t+\Delta t)&\approx \vec{v}(t) + \Delta t\,\vec{a}_i(t)
\end{align}
To close the expressions, we need an expression for the force in terms of the position and velocity of the particles. Its this force expression which sets the type of atom/molecule you are simulating.

In the example below toy spring particles are used. These are particles that push away from each other whenever they are closer than a certain distance $\sigma$

\begin{align}
    \vec{F}_{ij} = \begin{cases}
    -k (\sigma - \left|\vec{r}_{ij}\right|) \hat{r}_{ij} & \text{for $\left|\vec{r}_{ij}\right| < \sigma$}\\
    0 & \text{otherwise}
    \end{cases}
\end{align}
where $k$ is the spring stiffness setting how "hard" the particles are, $\vec{r}_{ij}=\vec{r}_i - \vec{r}_j$ is the vector pointing from particle $j$ to particle $i$, $\left|\vec{r}_{ij}\right|$ is the distance between the two particles, and $\hat{r}_{ij} = \vec{r}_{ij}/\left|\vec{r}_{ij}\right|$. The total force on a particle is just the sum of all the forces from other particles, as well as any global forces,

\begin{align}
    \vec{F}_{i} = \vec{F}_{body} + \sum_{i\neq j}\vec{F}_{ij}
\end{align}

The body force is any force like gravity which affects all particles and doesn't depend on the positions/velocities of other particles. Here, we just use another spring force that pulls all particles into the center of the system (to stop them flying away).

The actual integration algorithm used here is the velocity verlet algorithm, as its very good at conserving energy.

\begin{align}
\vec{r}_i &= \vec{r}_i + \frac{\Delta t}{2} \vec{v}_i\\
\vec{v}_i &= \vec{v}_i + \Delta t\, F_i / m_i\\
\vec{r}_i &= \vec{r}_i + \frac{\Delta t}{2} \vec{v}_i\\
\end{align}
where $m_i$ is the mass of the particle.

This is a great example of python's
power (and its lack of speed). Balance
<em>your</em> programming time against the <em>run
time</em> when choosing scripting or compiled
languages.

In [4]:
import numpy, math
# All the parameters of the simulation
box_length = 10 # How wide an area should the particles be scattered over
N = 20          # How many particles to simulate
dt = 0.1        # The timestep
k_particle = 5  # How stiff should the particle-particle interaction be
diameter = 2.0  # \sigma or the diameter of the particles
k_body = 0.1    # The stiffness of the spring holding the particles in the center of the sys

#Now we generate a Nx2 array/matrix holding the positions of all particles.
#They are randomly scattered from +- box_length
pos = (numpy.random.rand(N, 2) - 0.5) * box_length
#we also generate a Nx2 array of velocities initially at zero
vel = numpy.zeros((N,2))

#Calculate the force between two particles
def force(i, j):
    rij = pos[i] - pos[j]
    distance = numpy.linalg.norm(rij)
    if distance > diameter or i == j:
        # outside the interaction distance so return zero force
        return numpy.array([0,0])
    rij_norm = rij / distance
    return k_particle * (diameter - distance) * rij_norm

def body_force(i):
    return - k_body * pos[i]

def timestep(step):
    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]
    #pos += 0.5 * dt * vel
    
    forces = numpy.zeros((N,2))
    for i in range(N):
        forces[i] += body_force(i)
        for j in range(N):
            forces[i] += force(i,j)

    for i in range(N):
        vel[i] += dt * forces[i]
    #vel += dt * forces
    
    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]
    
    scatter.set_offsets(pos)
    return [scatter]

from IPython.display import HTML
import numpy as np
from matplotlib import animation
import matplotlib.pyplot as plt

plt.clf() #Clear the figure, as the animator leaves particles around if the cell is reurn
fig = plt.gcf()
ax = plt.gca()
ax.set_xlim(-box_length, +box_length)
ax.set_ylim(-box_length, +box_length)
scatter = ax.scatter(pos[:,0], pos[:,1], s=250)
print("Running calclations, please wait!")
anim = animation.FuncAnimation(fig, timestep, np.arange(1, 500), interval=10, blit=True)
HTML(anim.to_html5_video())

Running calclations, please wait!


The example code above contains some magic to render the simulation into a video, so that it can be viewed in the jupyter notebook. If you want to view the demo outside jupyter, a different version is given below

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
# parameters
boxsize = 10
N = 20
dt = 0.1        # timestep
k_particle = 5  # P-P stiffness
diameter = 2.0  # P diameter \sigma
k_body = 0.1    # stiffness of spring holding Ps in center of sys


# generate Nx2 array-matrix - positions of particles within box
pos = (np.random.rand(N, 2) - 0.5) * boxsize
vel = np.zeros((N,2))

# calculate force between 2 particles
def force(i, j):
    rij = pos[i] - pos[j]
    distance = np.linalg.norm(rij)
    if distance > diameter or i == j:
        return np.array([0,0])
    rij_norm = rij / distance
    return k_particle * (diameter - distance) * rij_norm

def body_force(i):
    return - k_body * pos[i]

def timestep(step):
    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]

    forces = np.zeros((N,2))
    for i in range(N):
        forces[i] += body_force(i)
        for j in range(N):
            forces[i] += force(i,j)

    for i in range(N):
        vel[i] += dt * forces[i]

    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]
    
    scatter.set_offsets(pos)
    plt.draw()
    plt.pause(0.01)


print("Running calculations, please wait!")
plt.clf()
plt.xlim(-boxsize, +boxsize)
plt.ylim(-boxsize, +boxsize)
scatter = plt.scatter(pos[:,0], pos[:,1], s=250)
for i in range(500):
    timestep(i)

Running calculations, please wait!
